### Program to get the property data from Shobha properties and store in vector db.

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

In [1]:
from langchain_community.document_loaders import WebBaseLoader,TextLoader
from langchain_community.vectorstores import Chroma
from langchain_chroma import Chroma
from langchain_openai import AzureOpenAIEmbeddings 
from langchain_text_splitters import RecursiveCharacterTextSplitter,TextSplitter
import requests
from bs4 import BeautifulSoup
import re

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [ ]:
url="https://danubeproperties.com/sitemap/"
response = requests.get(url)
soup=BeautifulSoup(response.text,"html.parser")
def find_all_elements_by_porfolio(href):
    return href and re.compile("portfolio").search(href)

links_1=[]
latest_projects=soup.find_all("a", attrs={"href": re.compile(".com/portfolio")})
# print(latest_projects)
for project in latest_projects:
    links_1.append(project.attrs.get('href'))

links_1

In [ ]:
def find_all_elements_by_class(class_):
        class_text='accordion-item'
        return class_ and re.compile(class_text).search(class_)
file = open("output.txt", "a",encoding='utf-8')
for url in links_1:
    response = requests.get(url)
    soup=BeautifulSoup(response.text,"html.parser")
    faqs=soup.find_all('div',class_=find_all_elements_by_class)
    for faq in faqs:
         file.writelines(faq.button.text)
         file.writelines(faq.div.text)

In [ ]:
text_loader = TextLoader("output.txt",encoding="utf-8")
text_docs=text_loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
doc_splits = text_splitter.split_documents(text_docs)

## Add all these text to vectordb

embedding=AzureOpenAIEmbeddings(api_key=os.getenv("AZURE_EMB_API_KEY"),
                                azure_endpoint=os.getenv("AZURE_EMB_ENDPOINT"))
vectorstoredb=Chroma.from_documents(
    documents=doc_splits,
    embedding=embedding,persist_directory="./databases/shobha_properties_chroma_db")